# Análise de vendas no festival indiano Diwali

O festival diwali, também conhecido como festival das luzes, é conhecido por sua grande movimentação comercial, por conta das festividades e troca de presentes.

O presente relatório tem o objetivo de nos trazer o perfil demográfico de maior impacto durante o festival, além de apresentar as categorias de produtos mais vendidos.

Os dados de vendas durante o festival podem ser encontrados na plataforma kaggle: https://www.kaggle.com/datasets/saadharoon27/diwali-sales-dataset

Os dados do censo da Índia no ano de 2011 pode ser encontrados no link: https://www.kaggle.com/datasets/shiivvvaam/indian-districts-population-data/data

### Importando bibliotecas e dataset

In [366]:
import pandas as pd
import numpy as np
import plotly.express as px

In [367]:
vendas_diwali = pd.read_csv('dataset\\diwali_sales.csv')
censo_indiano_2011 = pd.read_csv('dataset\\census_2011.csv')

### Conferindo e consertando quaisquer inconsistências - vendas_diwali

In [368]:
vendas_diwali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11251 entries, 0 to 11250
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        11251 non-null  int64  
 1   User_ID           11251 non-null  int64  
 2   Cust_name         11251 non-null  object 
 3   Product_ID        11251 non-null  object 
 4   Gender            11251 non-null  object 
 5   Age Group         11251 non-null  object 
 6   Age               11251 non-null  int64  
 7   Marital_Status    11251 non-null  int64  
 8   State             11251 non-null  object 
 9   Zone              11251 non-null  object 
 10  Occupation        11251 non-null  object 
 11  Product_Category  11251 non-null  object 
 12  Orders            11251 non-null  int64  
 13  Amount            11239 non-null  float64
 14  age_group         11115 non-null  object 
dtypes: float64(1), int64(5), object(9)
memory usage: 1.3+ MB


In [369]:
#'Columns Unamed: 0' foi deletada porque era apenas um índice que virou coluna durante o carregamento de dados 
vendas_diwali = vendas_diwali.drop(columns = 'Unnamed: 0')

In [370]:
#Trocando F e M por Feminino e Masculino
vendas_diwali['Gender'] = vendas_diwali['Gender'].replace({'F':'Feminino','M':'Masculino'})

In [371]:
#Informação sobre o dataset de venda_diwali
vendas_diwali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11251 entries, 0 to 11250
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   User_ID           11251 non-null  int64  
 1   Cust_name         11251 non-null  object 
 2   Product_ID        11251 non-null  object 
 3   Gender            11251 non-null  object 
 4   Age Group         11251 non-null  object 
 5   Age               11251 non-null  int64  
 6   Marital_Status    11251 non-null  int64  
 7   State             11251 non-null  object 
 8   Zone              11251 non-null  object 
 9   Occupation        11251 non-null  object 
 10  Product_Category  11251 non-null  object 
 11  Orders            11251 non-null  int64  
 12  Amount            11239 non-null  float64
 13  age_group         11115 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 1.2+ MB


In [372]:
#Conferindo as diferenças entre duas colunas similares.
#A coluna 'Age Group' possui mais opções de agrupamento do que a coluna 'age_group', alem de não conter valores ausentes.
print(vendas_diwali['Age Group'].unique())
print(vendas_diwali['age_group'].unique())

['26-35' '0-17' '18-25' '55+' '46-50' '51-55' '36-45']
['19-30' '31-40' '11-18' '51-80' '41-50' nan]


In [373]:
#Remoção da coluna 'age_group'
vendas_diwali = vendas_diwali.drop(columns = 'age_group')

In [374]:
#Convertendo dados binários em dados categóricos
vendas_diwali['Marital_Status'] = vendas_diwali['Marital_Status'].replace({0:'Solteiro',1:'Casado'})

In [375]:
#Tradução de colunas EN-PTBR
vendas_diwali = vendas_diwali.rename(
    columns = {
        'User_ID':'user_id',
        'Cust_name':'cliente_nome',
        'Product_ID':'id_do_produto',
        'Gender':'genero',
        'Age Group':'faixa_etaria',
        'Age':'idade',
        'Marital_Status':'estado_civil',
        'State':'estado',
        'Zone':'zona',
        'Occupation':'ocupacao',
        'Product_Category':'categoria_produto',
        'Orders':'pedidos',
        'Amount':'total',
        }
    )

### Conferindo e consertando quaisquer inconsistências - censo_indiano_2011

In [376]:
censo_indiano_2011.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ranking     610 non-null    int64  
 1   District    610 non-null    object 
 2   State       610 non-null    object 
 3   Population  610 non-null    object 
 4   Growth      610 non-null    object 
 5   Sex-Ratio   610 non-null    int64  
 6   Literacy    610 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 33.5+ KB


In [377]:
censo_indiano_2011.sample()

,Ranking,District,State,Population,Growth,Sex-Ratio,Literacy
424,455,Morigaon,Assam,"957,423",23.34 %,967,68.03


In [378]:
#Retirando vírgulas dos valores de população e transformando a string de população em uma string float
censo_indiano_2011['Population'] = censo_indiano_2011['Population'].str.replace(',', '').astype(float)


In [379]:
#Retirando o símbolo de percentual da coluna Growth e transformando em integral
censo_indiano_2011['Growth'] = censo_indiano_2011['Growth'].astype(str)
censo_indiano_2011['Growth'] = censo_indiano_2011['Growth'].str.replace('%', '').astype(int)

ValueError: invalid literal for int() with base 10: '36.01 '

In [ ]:
censo_indiano_2011 = censo_indiano_2011.rename(columns={
    'Ranking':'rank',
    'District':'distrito',
    'State':'estado',
    'Population':'populacao',
    'Growth':'crescimento',
    'Sex-Ratio':'razao_de_genero',
    'Literacy':'alfabetismo'
})

In [ ]:
#rank: A posição do distrito baseada na população
#distrito: O nome do distrito
#estado: Estado
#populacao: População total
#crescimento: O percentual de crescimento populacional
#razao_de_genero: A razão entre o número de homens para mulheres no distrito - x mulheres para 1000 homens
#alfabetismo: Taxa de alfabestismo no distrito

### Agrupamento de dados

In [ ]:
#Criação de agrupamento para identificar total de vendas por faixa etária
total_gasto_faixa_etaria = vendas_diwali.groupby(['faixa_etaria','genero'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por idade
total_gasto_idade = vendas_diwali.groupby(['idade','genero'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por gênero
total_gasto_genero = vendas_diwali.groupby(['genero','estado'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por estado civil
total_gasto_estado_civil = vendas_diwali.groupby(['estado_civil'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por categoria
total_gasto_categoria = vendas_diwali.groupby(['categoria_produto'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por Estado
total_gasto_estado = vendas_diwali.groupby(['estado','genero'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por Ocupação
total_gasto_ocupacao = vendas_diwali.groupby(['ocupacao'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

In [ ]:
#Criação de agrupamento para identificar total de vendas por Zona
total_gasto_zona = vendas_diwali.groupby(['zona'])['total'].sum().reset_index(name = 'total_gasto').sort_values(by = 'total_gasto',ascending = False)

### Apresentação de dados

In [ ]:
#Alteração do tipo de dado na coluna genero

total_gasto_faixa_etaria['genero'] = total_gasto_faixa_etaria['genero'].astype('category')

#Criação de gráfico de colunas para identificar total de vendas por faixa etária

bar_fig_por_faixa_etaria = px.bar(
    total_gasto_faixa_etaria,
    x='faixa_etaria',
    y='total_gasto',
    text_auto=".2s",
    color='genero',
    barmode = 'group'
)

# Atualização de aspectos visuais do gráfico
bar_fig_por_faixa_etaria.update_traces(
    textposition='outside'
)
bar_fig_por_faixa_etaria.update_yaxes(title_text='Total Gasto')
bar_fig_por_faixa_etaria.update_xaxes(title_text='Faixa Etária')
bar_fig_por_faixa_etaria.update_layout(title_text='Apresentação de total de vendas por faixa etária e gênero')

#Impressão do gráfico
bar_fig_por_faixa_etaria.show()

De acordo com os dados representados no gráfico acima, podemos identificar que o público feminino é responsável pela maior parte do total de gastos durante o período do festival. Além disso, o principal grupo de compradores está entre as mulheres e homens entre 26 a 35 anos.

Abaixo nós temos um gráfico representando a população por gênero de cada Estado Indiano.

In [ ]:
# Média razão de gênero por estado
media_razao_genero = censo_indiano_2011.groupby('estado')['razao_de_genero'].mean().reset_index()

#Coluna para separar o gráfico por cores
media_razao_genero['cor'] = media_razao_genero['razao_de_genero'] >= 1000

In [ ]:
#Criação de histograma para identificar a razão da populaçao do sexo feminino por
# masculino em cada Estado Indiano.

bar_fig_razao_genero = px.histogram(
    media_razao_genero,
    x='estado',
    y='razao_de_genero',
    text_auto=".3s",
    height=600,
    width=950,
    color='cor',
    labels={'cor': 'Razão de Gênero ≥ 1000'}
)

# Atualização de aspectos visuais do gráfico
bar_fig_razao_genero.update_traces(
    textposition='outside'
)
bar_fig_razao_genero.update_yaxes(title_text='Razão de mulheres a cada 1000 homens')
bar_fig_razao_genero.update_xaxes(title_text='Estado',tickangle = 45)
bar_fig_razao_genero.update_layout(title_text='Razão populacional entre homens e mulheres por Estado')

#Impressão do gráfico
bar_fig_razao_genero.show()

Acima é possível identificar que apenas 3 estados indianos possuem uma população feminina majoritária, esses seriam Uttarakhand, Puducherry e Kerala, abaixo foi realizada uma análise para verificar se, a presença majoritária da população indiana acarreta em algum impacto significativo no resultado final.

In [ ]:
#Junção do dataframe de total de gastos por genero (o qual inclui estado)
#com o dataframe do censo indiano com a razao de genero entre homens e mulheres em estados indianos
total_gasto_genero_razao=total_gasto_genero.merge(media_razao_genero,on='estado')

#Renomear coluna de color para identificar se a informação de que o estado possui mais mulheres que homens é
#verdadeira ou falsa
total_gasto_genero_razao = total_gasto_genero_razao.rename(columns={'cor':'mais_mulheres'})

In [ ]:
#Dataframe com o maior valor de gastos totais por cada Estado
df_max_total_gasto = total_gasto_genero_razao.loc[total_gasto_genero_razao.groupby('estado')['total_gasto'].idxmax()]

In [ ]:
#Identificação de genêros que fazer parte do dataframe com o maior valor de gastos totais por cada Estado
df_max_total_gasto['genero'].unique()

array(['Feminino'], dtype=object)

Acima podemos identificar que, em todos os estados, a população feminina foi responsável pela maior quantidade de gastos durante o festival.

In [ ]:
#Criação de histograma para identificar o valor total gasto pela população feminina em cada Estado
#O gráfico possui legenda para informar se a população daquele estado é de maioria feminina ou masculina

bar_fig_total_gasto = px.histogram(
    df_max_total_gasto,
    x='estado',
    y='total_gasto',
    text_auto=".3s",
    height=600,
    width=800,
    color='mais_mulheres',
    labels={'mais_mulheres': 'População majoritária feminina'}
)

# Atualização de aspectos visuais do gráfico
bar_fig_total_gasto.update_traces(
    textposition='outside'
)
bar_fig_total_gasto.update_yaxes(title_text='Soma do total de gastos')
bar_fig_total_gasto.update_xaxes(title_text='Estado')
bar_fig_total_gasto.update_layout(title_text='Total gasto por Estado - Presença majoritária de população feminina ou masculina')

#Impressão do gráfico
bar_fig_total_gasto.show()

O gráfico acima mostra que os dois Estados com maioria populacional feminina (Kerala e Uttarakhand) não correspondem com os Estados com maior total de vendas, 

In [ ]:
censo_indiano_2011

,rank,distrito,estado,populacao,crescimento,razao_de_genero,alfabetismo
0,1,Thane,Maharashtra,11060148.0,36.01 %,886,84.53
1,2,North Twenty Four Parganas,West Bengal,10009781.0,12.04 %,955,84.06
2,3,Bangalore,Karnataka,9621551.0,47.18 %,916,87.67
3,4,Pune,Maharashtra,9429408.0,30.37 %,915,86.15
4,5,Mumbai Suburban,Maharashtra,9356962.0,8.29 %,860,89.91
...,...,...,...,...,...,...,...
605,636,Nicobars,Andaman and Nicobar Islands,36842.0,-12.42 %,777,78.06
606,637,Upper Siang,Arunachal Pradesh,35320.0,5.87 %,889,59.99
607,638,Lahul and Spiti,Himachal Pradesh,31564.0,-5.00 %,903,76.81
608,639,Anjaw,Arunachal Pradesh,21167.0,14.19 %,839,56.46


In [ ]:
censo_por_estado_populacao = censo_indiano_2011.groupby('estado')['populacao'].sum().reset_index()

In [ ]:
censo_indiano_2011.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             610 non-null    int64  
 1   distrito         610 non-null    object 
 2   estado           610 non-null    object 
 3   populacao        610 non-null    float64
 4   crescimento      610 non-null    object 
 5   razao_de_genero  610 non-null    int64  
 6   alfabetismo      610 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 33.5+ KB


In [ ]:
censo_indiano_2011[censo_indiano_2011['estado']=='Andhra Pradesh']

,rank,distrito,estado,populacao,crescimento,razao_de_genero,alfabetismo
16,17,Rangareddy,Andhra Pradesh,5296741.0,48.16 %,961,75.87
18,19,East Godavari,Andhra Pradesh,5154296.0,5.16 %,1006,70.99
21,22,Guntur,Andhra Pradesh,4887813.0,9.47 %,1003,67.40
33,34,Krishna,Andhra Pradesh,4517398.0,7.87 %,992,73.74
43,44,Visakhapatnam,Andhra Pradesh,4290589.0,11.96 %,1006,66.91
46,47,Chittoor,Andhra Pradesh,4174064.0,11.43 %,997,71.53
51,52,Anantapur,Andhra Pradesh,4081148.0,12.10 %,977,63.57
53,54,Kurnool,Andhra Pradesh,4053463.0,14.85 %,988,59.97
54,55,Mahbubnagar,Andhra Pradesh,4053028.0,15.34 %,977,55.04
59,60,Hyderabad,Andhra Pradesh,3943323.0,2.97 %,954,83.25


In [ ]:
censo_por_estado_populacao

,estado,populacao
0,Andaman and Nicobar Islands,380581.0
1,Andhra Pradesh,67459740.0
2,Arunachal Pradesh,1383727.0
3,Assam,28381808.0
4,Bihar,89751364.0
5,Chandigarh,1055450.0
6,Chhattisgarh,25545198.0
7,Dadra and Nagar Haveli,343709.0
8,Daman and Diu,243247.0
9,Delhi,14056012.0
